In [1]:
import pandas as pd
import numpy as np

In [2]:
behav_loading = pd.read_csv('data/python_entropy_analysis/behavior_loading.csv', sep=',')
dis_loading = pd.read_csv('data/python_entropy_analysis/disease_loading.csv',sep=',')

In [11]:
pos_behav_loading = np.zeros((len(behav_loading),len(behav_loading.columns)-2))

# zero negative values
column_val = 0
for j in behav_loading.columns[2:len(behav_loading.columns)]:
    for i in range(0,len(behav_loading)):
        if behav_loading.iloc[i][j] > 0:
            pos_behav_loading[i,column_val] = behav_loading.iloc[i][j]
        else: 
            pos_behav_loading[i,column_val] = 0
    column_val = column_val + 1

column_sums = []
for j in range(0,pos_behav_loading.shape[1]):
        column_sums.append(np.sum(pos_behav_loading[:,j]))

# calculate probability as percentage
percentage_behav =  np.zeros((len(behav_loading),len(behav_loading.columns)-2))
for j in range(0,pos_behav_loading.shape[1]):
    for i in range(0,len(behav_loading)):
            percentage_behav[i,j] = pos_behav_loading[i][j]/column_sums[j]
    column_val = column_val + 1

# calculate entropy
p_ln_p =  -percentage_behav*np.log(percentage_behav)
behav_entropy = []
for j in range(0,pos_behav_loading.shape[1]):
        behav_entropy.append(np.nansum(p_ln_p[:,j]))
        
max_behav_entropy = behav_entropy/np.log(len(behav_loading))

d = {'Names': behav_loading.columns[2:len(behav_loading.columns)], 
     'B.Entropy': max_behav_entropy}
df = pd.DataFrame(data=d)
df

/Users/thomasvanasse/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log
/Users/thomasvanasse/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply


,Names,B.Entropy
0,IC-18,0.754075
1,IC-14,0.818712
2,IC-6,0.816269
3,IC-16,0.908787
4,IC-7,0.593518
5,IC-20,0.590506
6,IC-17,0.903849
7,IC-4,0.920880
8,IC-9,0.921451
9,IC-13,0.668529


In [12]:
pos_dis_loading = np.zeros((len(dis_loading),len(dis_loading.columns)-2))

# zero negative values
column_val = 0
for j in dis_loading.columns[2:len(dis_loading.columns)]:
    for i in range(0,len(dis_loading)):
        if dis_loading.iloc[i][j] > 0:
            pos_dis_loading[i,column_val] = dis_loading.iloc[i][j]
        else: 
            pos_dis_loading[i,column_val] = 0
    column_val = column_val + 1

column_sums = []
for j in range(0,pos_dis_loading.shape[1]):
        column_sums.append(np.sum(pos_dis_loading[:,j]))
    
# calculate probability as percentage
percentage_dis =  np.zeros((len(dis_loading),len(dis_loading.columns)-2))
for j in range(0,pos_dis_loading.shape[1]):
    for i in range(0,len(dis_loading)):
            percentage_dis[i,j] = pos_dis_loading[i][j]/column_sums[j]
    column_val = column_val + 1

# calculate entropy
p_ln_p =  -percentage_dis*np.log(percentage_dis)
dis_entropy = []
for j in range(0,pos_dis_loading.shape[1]):
        dis_entropy.append(np.nansum(p_ln_p[:,j]))
        
max_dis_entropy = dis_entropy/np.log(len(dis_loading))

d = {'Names': dis_loading.columns[2:len(dis_loading.columns)], 
     'A.Entropy': max_dis_entropy}
df = pd.DataFrame(data=d)
df
    

/Users/thomasvanasse/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log
/Users/thomasvanasse/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply


,Names,A.Entropy
0,IC-1,0.823457
1,IC-2,0.685278
2,IC-5,0.679060
3,IC-15,0.761087
4,IC-3,0.845814
5,IC-11,0.802837
6,IC-20,0.562115
7,IC-9,0.763281
8,IC12,0.861328
9,IC-7,0.567918
